Integrating the SceneGraph notebooks with the Toybot notebooks

This includes code to run the bot as script.

SceneGraph
==

The scenegraph stores operations as nodes.

These include that change state (fill, stroke etc) and operations that result in elements being drawn (path)

In [ ]:
class SceneGraph:
    def __init__(self):
        self.nodes = []
        
    def add_node(self, node):
        self.nodes.append(node)
    
    def __iter__(self):
        """
        Iterator that yields every node in the graph
        """
        yield from self.nodes.__iter__()

## Nodes

### Path

We'll start with the simplest shape -- a path made of straight lines -- and later implement new ones once we agree on a clear structure for the node/graph architecture.

It should only need:

* the minimum number of vars to store values
* methods to edit the path
* a method to represent the path as in the repl to help with debugging.

We will probably want a `Node` base class, but let's start simple.

In [ ]:
class Path:
    def __init__(self, coords=None):
        if coords is None:
            self.coords = []
        else:
            self.coords = coords
    
    def add_point(self, x, y):
        self.coords.append((x, y))

    def __repr__(self):
        return f'<Path: {self.coords}>'

Let's try it now:

In [ ]:
p = Path()
p.add_point(10, 10)
p.add_point(10, 100)
p.add_point(100, 100)
p.add_point(100, 10)

In [ ]:
print(p)

Now it's time to create an instance of our scene graph and add this path to it, so we can render it later.

In [ ]:
graph = SceneGraph()
graph.add_node(p)

ConsoleRenderer
==

The ConsoleRenderer outputs every node of scenegraph to the console.

JSON is the default output format, to make data transfer easy.

Data formats are changed by passing a function to `outputformat` that accepts nodes and returns data in the desired format.


In [ ]:
import jsons

In [ ]:
class ConsoleRenderer:
    def __init__(self, outputformat=jsons.dumps):
        if not outputformat:
            outputformat = lambda data: data
        self.outputformat=outputformat
    
    def render(self, graph):
        print(self.outputformat([node for node in graph]))

Now render our graph:

In [ ]:
renderer = ConsoleRenderer()
renderer.render(graph)

If JSON is not required, setting `outputformat` to None will output the scene graph content in the format used by the python repl:

In [ ]:
renderer = ConsoleRenderer(outputformat=None)
renderer.render(graph)

Grammar
==

The ToyBot class provides the user-facing API for drawing and setting up colours.

In [ ]:
class ToyBot:
    def __init__(self, graph):
        self.graph = graph
    
    def rect(self, x, y, width, height, fill=None, stroke=None):
        p = Path(coords=[(x, y),
                         (x, y+height),
                         (x+width, y+height),
                         (x+width, y)])
        self.graph.add_node(p)
        return p        

# Scripting

## Setup the scripting namespace

This function adds all the user-facing API of a bot into a namespace to enable scripting.

In [ ]:
import inspect

In [ ]:
def create_scripting_namespace(bot):
    namespace = {}
    for name, method in inspect.getmembers(bot, predicate=inspect.ismethod):
        if name.startswith('__'):
            continue
        namespace[name] = method
    return namespace

## Running a script

The run function accepts a bot script, does the setup needed to render a bot and then renders.

In [ ]:
def run(source):
    renderer = ConsoleRenderer()
    
    graph = SceneGraph()
    bot = ToyBot(graph)

    namespace = create_scripting_namespace(bot)
    
    exec(source, namespace)

    renderer.render(graph)

## Putting it all together - using the scripting interface

The scripting interface provides the simplest way of using ToyBot.

Pass the code to render to run function.

In [ ]:
code = """
rect(0, 0, 100, 100)
rect(25, 30, 50, 70)
"""

In [ ]:
run(code)